In [1]:
import json
import re
import pandas as pd
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Sentiment-Analysis").getOrCreate()
print(spark)

In [2]:
df =  spark.createDataFrame(pd.read_json('climatechange-v2.json'))   

In [3]:
from nltk.corpus import twitter_samples

In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)


positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                    for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                    for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [6]:
analysis=[]
for i in range(0,df.count()):
    custom_tweet=df.collect()[i]["text"]
    custom_tokens=remove_noise(word_tokenize(custom_tweet))
    analysis.append(classifier.classify(dict([token, True] for token in custom_tokens)))
    
res = []
for i in analysis:
    sub = i.split(', ')
    res.append(sub)

NameError: name 'columns' is not defined

In [7]:
columns=["Sentiment"] 
df_a = spark.createDataFrame(res, columns)
df_a.show()

+---------+
|Sentiment|
+---------+
| Positive|
| Negative|
| Positive|
| Positive|
| Positive|
| Positive|
| Positive|
| Negative|
| Negative|
| Positive|
+---------+



In [10]:
df.withColumn("Sentiment",df_a.Sentiment)

AnalysisException: Resolved attribute(s) Sentiment#38 missing from id#0L,username#1,text#2,favourite_count#3L,retweet_count#4L,source#5,location#6 in operator !Project [id#0L, username#1, text#2, favourite_count#3L, retweet_count#4L, source#5, location#6, Sentiment#38 AS Sentiment#60].;
!Project [id#0L, username#1, text#2, favourite_count#3L, retweet_count#4L, source#5, location#6, Sentiment#38 AS Sentiment#60]
+- LogicalRDD [id#0L, username#1, text#2, favourite_count#3L, retweet_count#4L, source#5, location#6], false


In [11]:
merged_df = df.unionByName(df_a, allowMissingColumns=True)

In [13]:
df2=merged_df.toPandas()

In [14]:
df2

,id,username,text,favourite_count,retweet_count,source,location,Sentiment
0,1.432970e+18,JFSebastian146,RT @lavergnetho: @AschbacherJosef @IPCC_CH @es...,0.0,10.0,My1stNewApp,,None
1,1.432970e+18,SAFER_Lab,RT @JacobEurich: Accepting abstracts for our O...,0.0,15.0,Twitter Web App,"Rhodes University, RSA",None
2,1.432970e+18,XylemWater,RT @WaterUK: 🌍Everyone can help fight #Climate...,0.0,6.0,Twitter Web App,United Kingdom,None
3,1.432970e+18,Lorenz41294196,👋 @SenSchumer @SpeakerPelosi thank you for you...,0.0,0.0,Twitter for Android,,None
4,1.432970e+18,_thebastion_,Urgent decisive action at the intersection of ...,0.0,0.0,Twitter Web App,New Delhi,None
5,1.432970e+18,PopCounts,RT @PopnMatters: A bit unsure of the links bet...,0.0,9.0,Twitter for Android,Global,None
6,1.432970e+18,Farozaankhi,RT @UrbanPlannerNED: Glad to see it as #Faroza...,0.0,2.0,Twitter Web App,Karachi,None
7,1.432970e+18,JFSebastian146,RT @ProfStrachan: Sixty years of #ClimateChang...,0.0,166.0,My1stNewApp,,None
8,1.432970e+18,Stephen31626808,RT @Love_plants: Thread: @IanPlantlife on @bgc...,0.0,4.0,Twitter Web App,,None
9,1.432970e+18,iamgabbarson,RT @TPD_Journal: Featuring #OpenAccess #Resear...,0.0,1.0,Hey OA,New Jersey,None
